In [7]:
!pip install tensorflow==2.4.0
!pip install keras==2.2.4

In [8]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [9]:
import numpy as np

In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.models import load_model

In [11]:
train_datagen=ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)

In [12]:
test_datagen=ImageDataGenerator(rescale=1./255)

In [13]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.

if os.environ.get('RUNTIME_ENV_LOCATION_TYPE') == 'external':
    endpoint_9850a50ebeed4c27a806eecbd24caac9 = 'https://s3.us.cloud-object-storage.appdomain.cloud'
else:
    endpoint_9850a50ebeed4c27a806eecbd24caac9 = 'https://s3.private.us.cloud-object-storage.appdomain.cloud'

client_9850a50ebeed4c27a806eecbd24caac9 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='D7XYHfjV-ggYKGOqkw0WC_DXewfLXmSuOBN-oeZ7IAKB',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url=endpoint_9850a50ebeed4c27a806eecbd24caac9)

streaming_body_1 = client_9850a50ebeed4c27a806eecbd24caac9.get_object(Bucket='reciperecognitionproject-donotdelete-pr-53tbbsf5qgh4br', Key='dataset.zip')['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


In [14]:
from io import BytesIO
import zipfile
unzip=zipfile.ZipFile(BytesIO(streaming_body_1.read()),'r')
file_paths=unzip.namelist()
for path in file_paths:
    unzip.extract(path)

In [15]:
pwd


'/home/wsuser/work'

In [16]:
import os
filenames=os.listdir('/home/wsuser/work/dataset/training_set')

In [17]:
x_train=train_datagen.flow_from_directory(directory=r'/home/wsuser/work/dataset/training_set',target_size=(64,64),batch_size=32,class_mode='categorical')

Found 2418 images belonging to 3 classes.


In [18]:
x_test=test_datagen.flow_from_directory(directory=r'/home/wsuser/work/dataset/test_set',target_size=(64,64),batch_size=32,class_mode='categorical')

Found 600 images belonging to 3 classes.


In [19]:
print(x_train.class_indices)

{'french_fries': 0, 'pizza': 1, 'samosa': 2}


In [20]:
from collections import Counter as c

In [21]:
c(x_train.labels)

Counter({0: 804, 1: 804, 2: 810})

In [22]:
model=Sequential()

In [23]:
model.add(Conv2D(32,(3,3),input_shape=(64,64,3),activation='relu'))

In [24]:
model.add(MaxPooling2D(pool_size=(2,2)))

In [25]:
model.add(Conv2D(32,(3,3),activation='relu'))

In [26]:
model.add(MaxPooling2D(pool_size=(2,2)))

In [27]:
model.add(Dropout(0.2))

In [28]:
model.add(Flatten())

In [29]:
model.add(Dense(units=32,activation='relu'))

In [30]:
model.add(Dense(units=3,activation='softmax'))

In [31]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 62, 62, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 31, 31, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 29, 29, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 14, 14, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 6272)              0         
_________________________________________________________________
dense (Dense)                (None, 32)                2

In [32]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [33]:
print(len(x_train),len(x_test))

76 19


In [34]:
model.fit(x_train,steps_per_epoch=len(x_train),epochs=80,validation_data=x_test,validation_steps=len(x_test))#steps_per_epoch=76,validation_steps=19

Epoch 1/80
76/76 [==============================] - 24s 304ms/step - loss: 1.0907 - accuracy: 0.3774 - val_loss: 0.9696 - val_accuracy: 0.5300
Epoch 2/80
76/76 [==============================] - 22s 291ms/step - loss: 0.9533 - accuracy: 0.5542 - val_loss: 0.9089 - val_accuracy: 0.5550
Epoch 3/80
76/76 [==============================] - 22s 293ms/step - loss: 0.8934 - accuracy: 0.5904 - val_loss: 0.8811 - val_accuracy: 0.5800
Epoch 4/80
76/76 [==============================] - 22s 289ms/step - loss: 0.8264 - accuracy: 0.6189 - val_loss: 0.7827 - val_accuracy: 0.6667
Epoch 5/80
76/76 [==============================] - 22s 291ms/step - loss: 0.7562 - accuracy: 0.6705 - val_loss: 0.7399 - val_accuracy: 0.6600
Epoch 6/80
76/76 [==============================] - 22s 291ms/step - loss: 0.7126 - accuracy: 0.6841 - val_loss: 0.7146 - val_accuracy: 0.6767
Epoch 7/80
76/76 [==============================] - 22s 293ms/step - loss: 0.7250 - accuracy: 0.6906 - val_loss: 0.6352 - val_accuracy: 0.7300

In [35]:
model.save('Model.h5')

In [36]:
!tar -zcvf recipe-recognition-model.tgz Model.h5

Model.h5


In [37]:
ls -1

dataset/
final-model.tar.gz
Model.h5
new-model.tar.gz
recipe-recognition-model.tgz


In [33]:
!pip install watson-machine-learning-client --upgrade

     |████████████████████████████████| 538 kB 15.7 MB/s eta 0:00:01


In [38]:
from ibm_watson_machine_learning import APIClient
wml_credentials={
                  "url":"https://us-south.ml.cloud.ibm.com",
                  "apikey":"C2p5ZNy2IeiwKkWQkCav1moEq0XiT6oaz0txuVAoq2QP"
}
client =APIClient(wml_credentials)

In [39]:
client=APIClient(wml_credentials)

In [40]:
def guid_from_space_name(client,space_name):
    space =client.spaces.get_details()
    return(next(item for item in space['resources'] if item['entity']["name"]==space_name)['metadata']['id'])

In [41]:
space_uid=guid_from_space_name(client,'recipe-recognition')
print("Space UID ="+space_uid)

Space UID =d473e596-fe8f-4d7d-9898-76726c258a8a


In [42]:
client.set.default_space(space_uid)

'SUCCESS'

In [43]:
client.software_specifications.list()

-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod   1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl      111e41b3-de2d-5422-a4d6-bf776828c4b7  base
scikit-learn_0.22-py3.6        154010fa-5b3b-4ac1-82af-4d5ee5abbc85  base
default_r3.6                   1b70aec3-ab34-4b87-8aa0-a4a3c8296a36  base
pytorch-onnx_1.3-py3.6         1bc6029

In [44]:
software_spec_uid=client.software_specifications.get_uid_by_name("tensorflow_2.4-py3.7")
software_spec_uid

'65e171d7-72d1-55d9-8ebb-f813d620c9bb'

In [45]:
model_details=client.repository.store_model(model='recipe-recognition-model.tgz',meta_props={
    client.repository.ModelMetaNames.NAME:"CNN",
    client.repository.ModelMetaNames.TYPE:"keras_2.2.4",
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_spec_uid
})
model_id=client.repository.get_model_uid(model_details)

Note: Warnings!! :  Model type keras_2.2.4 is deprecated. We recommend you use a supported model type. See Supported Frameworks https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/pm_service_supported_frameworks.html


In [46]:
model_id

'80096af2-31a2-43ed-893d-b9f0b2df75bb'

In [48]:
client.repository.download(model_id,'RR-model.tar.gz')

Successfully saved model content to file: 'RR-model.tar.gz'


'/home/wsuser/work/RR-model.tar.gz'

In [49]:
from tensorflow.keras.models import load_model
from keras.preprocessing import image
model=load_model("Model.h5")

In [50]:
!pip uninstall pillow -y

Found existing installation: Pillow 9.0.0
Uninstalling Pillow-9.0.0:
  Successfully uninstalled Pillow-9.0.0


In [51]:
!pip install pillow

  Using cached Pillow-9.0.0-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (4.3 MB)


In [54]:
import PIL
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import array_to_img

In [56]:

streaming_body_4 = client_9850a50ebeed4c27a806eecbd24caac9.get_object(Bucket='reciperecognitionproject-donotdelete-pr-53tbbsf5qgh4br', Key='pizza1.jpg')['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


In [61]:
from PIL import Image

In [63]:
img=Image.open(streaming_body_4)

In [65]:
img=img.resize((64,64))
x = image.img_to_array(img)
x = np.expand_dims(x,axis = 0)
pred = model.predict(x)
pred

array([[0., 1., 0.]], dtype=float32)

In [66]:
index=['french_fries', 'pizza', 'samosa']
result=np.argmax(pred,axis=1)
result

array([1])

In [67]:
result=index[result[0]]
result

'pizza'

In [69]:

streaming_body_5 = client_9850a50ebeed4c27a806eecbd24caac9.get_object(Bucket='reciperecognitionproject-donotdelete-pr-53tbbsf5qgh4br', Key='FF.jpg')['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


In [70]:
img=Image.open(streaming_body_5)
img=img.resize((64,64))
x = image.img_to_array(img)
x = np.expand_dims(x,axis = 0)
pred = model.predict(x)
pred

array([[1., 0., 0.]], dtype=float32)

In [71]:
index=['french_fries', 'pizza', 'samosa']
result=np.argmax(pred,axis=1)
result

array([0])

In [72]:
result=index[result[0]]
result

'french_fries'

In [73]:

streaming_body_6 = client_9850a50ebeed4c27a806eecbd24caac9.get_object(Bucket='reciperecognitionproject-donotdelete-pr-53tbbsf5qgh4br', Key='1976.jpg')['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


In [74]:
img=Image.open(streaming_body_6)
img=img.resize((64,64))
x = image.img_to_array(img)
x = np.expand_dims(x,axis = 0)
pred = model.predict(x)
pred

array([[0., 0., 1.]], dtype=float32)

In [75]:
index=['french_fries', 'pizza', 'samosa']
result=np.argmax(pred,axis=1)
result

array([2])

In [76]:
result=index[result[0]]
result

'samosa'